### This is a notebook created in Databricks - Rishi

Spark dataframe and visualizations may not be available for most outputs, as the current notebook was ran with a local spark session. 

In [47]:
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession

# from pyspark import SparkConf, SparkContext
from datetime import datetime, date, timedelta
from dateutil import relativedelta
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.functions import to_timestamp, to_date
from pyspark.sql import functions as F
from pyspark.sql.functions import collect_list, collect_set, concat, first, array_distinct, col, size, expr
import random
import warnings
warnings.filterwarnings('ignore')

In [26]:
#Start the spark session (Although it is not required if notebook directly ran in Databricks)
spark = SparkSession.builder \
    .appName("Flight Data Analysis in Spark") \
    .getOrCreate()

### Loading Dataset
[Airbnb San Francisco Listing](https://www.kaggle.com/datasets/jeploretizo/san-francisco-airbnb-listings)

In [27]:

file_path = "./listings.csv"

raw_df = spark.read.csv(file_path, header="true", inferSchema="true", multiLine="true", escape='"')

display(raw_df)

DataFrame[id: int, listing_url: string, scrape_id: double, last_scraped: string, name: string, summary: string, space: string, description: string, experiences_offered: string, neighborhood_overview: string, notes: string, transit: string, access: string, interaction: string, house_rules: string, thumbnail_url: string, medium_url: string, picture_url: string, xl_picture_url: string, host_id: int, host_url: string, host_name: string, host_since: string, host_location: string, host_about: string, host_response_time: string, host_response_rate: string, host_acceptance_rate: string, host_is_superhost: string, host_thumbnail_url: string, host_picture_url: string, host_neighbourhood: string, host_listings_count: int, host_total_listings_count: int, host_verifications: string, host_has_profile_pic: string, host_identity_verified: string, street: string, neighbourhood: string, neighbourhood_cleansed: string, neighbourhood_group_cleansed: string, city: string, state: string, zipcode: string, ma

In [28]:
raw_df.columns

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',


In [29]:
columns_to_keep = [
    "host_is_superhost",
    "cancellation_policy",
    "instant_bookable",
    "host_total_listings_count",
    "neighbourhood_cleansed",
    "latitude",
    "longitude",
    "property_type",
    "room_type",
    "accommodates",
    "bathrooms",
    "bedrooms",
    "beds",
    "bed_type",
    "minimum_nights",
    "number_of_reviews",
    "review_scores_rating",
    "review_scores_accuracy",
    "review_scores_cleanliness",
    "review_scores_checkin",
    "review_scores_communication",
    "review_scores_location",
    "review_scores_value",
    "price"
]

base_df = raw_df.select(columns_to_keep)
base_df.cache().count()
display(base_df)

24/05/01 17:41:03 WARN CacheManager: Asked to cache already cached data.


DataFrame[host_is_superhost: string, cancellation_policy: string, instant_bookable: string, host_total_listings_count: int, neighbourhood_cleansed: string, latitude: double, longitude: double, property_type: string, room_type: string, accommodates: int, bathrooms: double, bedrooms: int, beds: int, bed_type: string, minimum_nights: int, number_of_reviews: int, review_scores_rating: int, review_scores_accuracy: int, review_scores_cleanliness: int, review_scores_checkin: int, review_scores_communication: int, review_scores_location: int, review_scores_value: int, price: string]

In [30]:
#Fixing Data Types

from pyspark.sql.functions import col, translate

fixed_price_df = base_df.withColumn("price", translate(col("price"), "$,", "").cast("double"))

display(fixed_price_df)

DataFrame[host_is_superhost: string, cancellation_policy: string, instant_bookable: string, host_total_listings_count: int, neighbourhood_cleansed: string, latitude: double, longitude: double, property_type: string, room_type: string, accommodates: int, bathrooms: double, bedrooms: int, beds: int, bed_type: string, minimum_nights: int, number_of_reviews: int, review_scores_rating: int, review_scores_accuracy: int, review_scores_cleanliness: int, review_scores_checkin: int, review_scores_communication: int, review_scores_location: int, review_scores_value: int, price: double]

## Summary Statistics

In [31]:
display(fixed_price_df.describe())

DataFrame[summary: string, host_is_superhost: string, cancellation_policy: string, instant_bookable: string, host_total_listings_count: string, neighbourhood_cleansed: string, latitude: string, longitude: string, property_type: string, room_type: string, accommodates: string, bathrooms: string, bedrooms: string, beds: string, bed_type: string, minimum_nights: string, number_of_reviews: string, review_scores_rating: string, review_scores_accuracy: string, review_scores_cleanliness: string, review_scores_checkin: string, review_scores_communication: string, review_scores_location: string, review_scores_value: string, price: string]

In [32]:
display(fixed_price_df.summary())

DataFrame[summary: string, host_is_superhost: string, cancellation_policy: string, instant_bookable: string, host_total_listings_count: string, neighbourhood_cleansed: string, latitude: string, longitude: string, property_type: string, room_type: string, accommodates: string, bathrooms: string, bedrooms: string, beds: string, bed_type: string, minimum_nights: string, number_of_reviews: string, review_scores_rating: string, review_scores_accuracy: string, review_scores_cleanliness: string, review_scores_checkin: string, review_scores_communication: string, review_scores_location: string, review_scores_value: string, price: string]

In [33]:
#Getting rid of extreme values

display(fixed_price_df.select("price").describe())

DataFrame[summary: string, price: string]

In [34]:
fixed_price_df.filter(col("price")==0).count()

1

In [35]:
#Now only keep rows with a strictly positive price
pos_prices_df = fixed_price_df.filter(col("price") > 0)

In [36]:
# A minimum stay of one year seems to be a reasonable limit here. Let's filter out those records where the *minimum_nights* 
#is greater than 365.

min_nights_df = pos_prices_df.filter(col("minimum_nights") <= 365)

display(min_nights_df)

DataFrame[host_is_superhost: string, cancellation_policy: string, instant_bookable: string, host_total_listings_count: int, neighbourhood_cleansed: string, latitude: double, longitude: double, property_type: string, room_type: string, accommodates: int, bathrooms: double, bedrooms: int, beds: int, bed_type: string, minimum_nights: int, number_of_reviews: int, review_scores_rating: int, review_scores_accuracy: int, review_scores_cleanliness: int, review_scores_checkin: int, review_scores_communication: int, review_scores_location: int, review_scores_value: int, price: double]

## Handling Null Values

Some ways to handle nulls:
* Drop any records that contain nulls
* Numeric:
  * Replace them with mean/median/zero/etc.
* Categorical:
  * Replace them with the mode
  * Create a special category for null
* Use techniques like ALS (Alternating Least Squares) which are designed to impute missing values

In [37]:
# For imputing all numeric variables must be double 

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

integer_columns = [x.name for x in min_nights_df.schema.fields if x.dataType == IntegerType()]
doubles_df = min_nights_df

for c in integer_columns:
    doubles_df = doubles_df.withColumn(c, col(c).cast("double"))

columns = "\n - ".join(integer_columns)
print(f"Columns converted from Integer to Double:\n - {columns}")

Columns converted from Integer to Double:
 - host_total_listings_count
 - accommodates
 - bedrooms
 - beds
 - minimum_nights
 - number_of_reviews
 - review_scores_rating
 - review_scores_accuracy
 - review_scores_cleanliness
 - review_scores_checkin
 - review_scores_communication
 - review_scores_location
 - review_scores_value


In [41]:
# Add a dummy column to denote the presence of null values before imputing (i.e. 1.0 = Yes, 0.0 = No).

from pyspark.sql.functions import when, col

impute_cols = [
    "bedrooms",
    "bathrooms",
    "beds", 
    "review_scores_rating",
    "review_scores_accuracy",
    "review_scores_cleanliness",
    "review_scores_checkin",
    "review_scores_communication",
    "review_scores_location",
    "review_scores_value"
]

for c in impute_cols:
    doubles_df = doubles_df.withColumn(c + "_na", when(col(c).isNull(), 1.0).otherwise(0.0))


In [42]:
display(doubles_df.describe())

DataFrame[summary: string, host_is_superhost: string, cancellation_policy: string, instant_bookable: string, host_total_listings_count: string, neighbourhood_cleansed: string, latitude: string, longitude: string, property_type: string, room_type: string, accommodates: string, bathrooms: string, bedrooms: string, beds: string, bed_type: string, minimum_nights: string, number_of_reviews: string, review_scores_rating: string, review_scores_accuracy: string, review_scores_cleanliness: string, review_scores_checkin: string, review_scores_communication: string, review_scores_location: string, review_scores_value: string, price: string, bedrooms_na: string, bathrooms_na: string, beds_na: string, review_scores_rating_na: string, review_scores_accuracy_na: string, review_scores_cleanliness_na: string, review_scores_checkin_na: string, review_scores_communication_na: string, review_scores_location_na: string, review_scores_value_na: string]

## Transformers and Estimators

In [43]:
from pyspark.ml.feature import Imputer

imputer = Imputer(strategy="median", inputCols=impute_cols, outputCols=impute_cols)

imputer_model = imputer.fit(doubles_df)
imputed_df = imputer_model.transform(doubles_df)

In [46]:
# For saving as delta table
# imputed_df.write.format("delta").mode("overwrite").save(f"{DA.paths.working_dir}/imputed_results")

#For local, we'll store this as a csv file 

imputed_df.write.csv(path='./cleaned_listings.csv', mode='overwrite', header=True)
